In [16]:
from peak_resolver.linear_combination_fitting import *
from peak_resolver.get_models_bases import get_models
from peak_resolver.process_chromeleon import process_chromeleon_file
import pickle
import pathlib
from pathlib import Path
import xarray as xr
from xarray import apply_ufunc

In [20]:
def get_basis(root, conc, tmin, tmax, tgrid):
    p = Path(root).resolve()
    # print(list(files))

    if conc=='0.5':
        search = '*6_2*.txt'
    elif conc=='0.4':
        search = '*5_2*.txt'
    elif conc == '0.3':
        search = '*4_2*.txt'
    elif conc == '0.2':
        search = '*3_2*.txt'
    elif conc == '0.1':
        search = '*2_2*.txt'
    elif conc == '0.05':
        search = '*1_2*.txt'
    files = p.rglob(search)

    # print(*list(files))


    for file in list(files):
        print(file)
        data = process_chromeleon_file(file, tmin, tmax, tgrid = tgrid)
        print(data.name)
        
    return

In [21]:
ds

NameError: name 'ds' is not defined

In [22]:
basis_root = '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/CALIBRATION CURVES/'
p = get_basis(basis_root, '0.5', tmin = 6.5, tmax = 9, tgrid = np.linspace(0.15, 19.85, 5997))

/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/CALIBRATION CURVES/ACETIC ACID/A6_2023_11_17.txt


ValueError: Length of values (5997) does not match length of index (8999)

In [74]:
ds = xr.load_dataset(
    '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/2023-12-07.nc'
)
ds = ds.drop_vars('index')

In [79]:
ds

<xarray.Dataset>
Dimensions:    (t: 5997, exp_time: 5, run: 3, side: 2, Acetic: 2, Propionic: 2,
                Butyric: 2, Valeric: 2, Caproic: 2)
Coordinates:
  * t          (t) float64 0.15 0.1533 0.1566 0.1599 ... 19.84 19.84 19.85 19.85
  * exp_time   (exp_time) float64 0.0 1.0 2.0 3.0 4.0
  * run        (run) int32 1 2 3
  * side       (side) object 'F' 'R'
  * Acetic     (Acetic) int32 0 50
  * Propionic  (Propionic) int32 0 50
  * Butyric    (Butyric) int32 0 50
  * Valeric    (Valeric) int32 0 50
  * Caproic    (Caproic) int32 0 50
Data variables:
    s          (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    ds         (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...
    d2s        (side, exp_time, run, Acetic, Propionic, Butyric, Valeric, Caproic, t) float64 ...

In [87]:
def mean(data, dim="t"):

    return apply_ufunc(

        np.abs,

        data,

        input_core_dims=[dim],

        vectorize=True,

    )


    

In [88]:
mean = mean(ds['s'])

ValueError: setting an array element with a sequence.

In [83]:
mean

<function __main__.mean(data, dim='t')>

In [42]:
df = ds['ds'].to_dataframe().dropna()

In [43]:
df

ds
side exp_time run Acetic Propionic Butyric Valeric Caproic t                  
F    0.0      1   0      0         0       50      0       0.150000   0.000010
                                                           0.153286   0.000010
                                                           0.156571   0.000010
                                                           0.159857   0.000010
                                                           0.163142   0.000010
...                                                                        ...
R    4.0      3   0      0         0       50      0       19.836858 -0.000059
                                                           19.840143 -0.000068
                                                           19.843429 -0.000062
                                                           19.846714 -0.000076
                                                           19.850000 -0.000080

[659670 rows x 1 columns]

In [3]:
for side in ds.coords['side'].values:
    for run in ds.coords['run']:
        for exp_time in ds.coords['exp_time']:
            for ace in ds.coords['Acetic']:
                for pro in ds.coords['Propionic']:
                    for but in ds.coords['Butyric']:
                        for val in ds.coords['Valeric']:
                            for cap in ds.coords['Caproic']:
                                print(ds.sel(side=side, 
                                             run=run, 
                                             exp_time=exp_time, 
                                             Acetic=ac,
                                             Propionic=pro,
                                             Butyric=but,
                                             Valeric=val,
                                             Caproic=cap
                                            ).dropna(dim='t', how='all'))
                


NameError: name 'ac' is not defined

In [36]:
root = '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves'
acids = ['acetic', 'propanoic', 'butyric', 'valeric', 'caproic']
models = get_models(root, acids)
acids = [a.capitalize() for a in acids]
for i in list(models):
    models[i + '_err'] = np.sqrt(np.sum(models[i].resid)**2/models[i].df_resid)

acetic
propanoic
butyric
valeric
caproic


In [38]:
models['ace'].resid

array([-0.00071639, -0.00077775,  0.00157829,  0.00446897, -0.00843642,
        0.00388329])

In [37]:
models

{'ace': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15b186bd0>,
 'pro': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15ce37710>,
 'but': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15ce9a090>,
 'val': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15ce9bf50>,
 'cap': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15ce9a0f0>,
 'ace_err': 6.938893903907228e-18,
 'pro_err': 1.2836953722228372e-16,
 'but_err': 3.469446951953614e-18,
 'val_err': 3.9898639947466563e-16,
 'cap_err': 1.3877787807814457e-17}

In [94]:
a = xr.DataArray([[1,2,3], [4,5,6]], dims=['x', 'y'])
b = xr.DataArray([[-1, -2, -3], [4,5,6]], dims=['x', 'y'])

In [113]:
b

<xarray.DataArray (x: 2, y: 3)>
array([[-1, -2, -3],
       [ 4,  5,  6]])
Dimensions without coordinates: x, y

In [130]:
def magnitude(a, b):

    # func = lambda x, y: np.sqrt(x**2 + y**2)

    return xr.apply_ufunc(norm, a, b)

def norm(x, y):
    return np.sqrt(x**2 + y**2)

def _inner(x, y):

    result = np.matmul(x[..., np.newaxis, :], y[..., :, np.newaxis])

    return result[..., 0, 0]


def inner_product(a, b, dim):

    return apply_ufunc(_inner, a, b, input_core_dims=[[dim], [dim]])

def v_lin_com(vectors, dim='t', initial_guess=None, bounds = None, method='Nelder-Mead',):

    return apply_ufunc(lin_combination_fitting, target, vectors, initial_guess=initial_guess, bounds = None, method='Nelder-Mead', input_core_dims=[[dim], [dim]])


In [125]:
v_lin_com()

<xarray.DataArray (y: 3)>
array([15, 21, 27])
Dimensions without coordinates: y

In [126]:
a

<xarray.DataArray (x: 2, y: 3)>
array([[1, 2, 3],
       [4, 5, 6]])
Dimensions without coordinates: x, y

In [127]:
b

<xarray.DataArray (x: 2, y: 3)>
array([[-1, -2, -3],
       [ 4,  5,  6]])
Dimensions without coordinates: x, y

In [128]:
inner_product(a,b,'y')

<xarray.DataArray (x: 2)>
array([-14,  77])
Dimensions without coordinates: x